In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib tk

In [2]:
def entr(d):
    try:
        res = []
        EPS = 1e-10
        for v in d:
            if abs(v) < EPS or abs(1-v) < EPS:
                res.append(0)
            else:
                res.append(-v*np.log2(v) -(1-v)*np.log2(1-v))
        return np.array(res)
    except TypeError:
        return -d*np.log2(d) -(1-d)*np.log2(1-d)

def KL(p, q):
    if (p in [0,1]) or (q in [0,1]):
        return np.inf
    return p * np.log2(p/q) + (1-p) * np.log2((1-p)/(1-q))

In [3]:
df = pd.read_csv("upper_output_3D_n1000.csv")

In [8]:
vals = df.values
X = vals[:,0]
Y = vals[:,1]
Pi_vals = Y - entr(X)
plt.plot(X,Pi_vals)

In [7]:
plt.plot(X, Pi_vals + 2* entr(X) - 2*(1-X))

In [22]:
N = 1000
def get_UB(d, d_vals, Pis):
    vals=[]
    for dprime, Pi in zip(d_vals, Pis):
        if dprime >= d:
            continue
        delta = d - dprime
        vals.append(Pi + entr(d) + delta + 2**(-N*KL(dprime, d)) + 1/N * np.log2(2*np.pi*dprime*(1-dprime)*N) + 1/N * np.log2(8*np.pi*np.e*max(d*(1-d)*N, 1/6)) + np.log2(np.e) / (N**2 * 3 * min(dprime, 1-dprime)) )
    if len(vals) == 0:
        print("error")
    return min(vals) if len(vals) > 0 else 0

In [23]:
UB = np.array([get_UB(xi, X, Pi_vals) for xi in X])

error


<ipython-input-2-fb02ad7a8ad5>:12: RuntimeWarning: divide by zero encountered in log2
  return -d*np.log2(d) -(1-d)*np.log2(1-d)
<ipython-input-2-fb02ad7a8ad5>:12: RuntimeWarning: invalid value encountered in double_scalars
  return -d*np.log2(d) -(1-d)*np.log2(1-d)


In [32]:
shifted_UB = 1-(X +entr(X) -UB)
LB_TRIVIAL = (1-entr(X))*(X <= 1/2)
# plt.plot(X, shifted_UB - LB_TRIVIAL, label='diff')
plt.plot(X,shifted_UB, label="UB-shifted")
plt.plot(X, 1-(X +entr(X) -Y), label="UB-not-shifted")
plt.plot(X, LB_TRIVIAL, label="lb-trivial")
plt.plot()


df_sim = pd.read_csv("sim_output_n10000_low_iters.csv")
vals_sim = df_sim.values
X_sim = vals_sim[:,0]
Y_sim = vals_sim[:,1]
plt.plot(X_sim, 1-(X_sim +entr(X_sim) -Y_sim))

# plt.plot(X_sim, (1-(X_sim +entr(X_sim) -Y_sim)) - (1-(X +entr(X) -UB)) )
# plt.plot(X, X*0+1)
plt.legend()
plt.show()

In [64]:
df_unif = pd.read_csv("upper_output.csv")
vals_unif = df_unif.values
X_unif = vals_unif[:,0]
Y_unif = vals_unif[:,1]
plt.plot(X_unif, X_unif +entr(X_unif) -Y_unif)

In [47]:
def get_min_vals(d):
    min_val = 100
    min_val_dprime = 0
    min_val_term1 = 0
    min_val_term2 = 0
    min_val_term3 = 0
    min_val_term4 = 0
    min_val_term5 = 0

    for dprime in np.linspace(0.001,d-0.001,100):
        delta = d- dprime
        term1 = delta
        term2 = 2**(-N*KL(d, dprime))
        term3 = 1/N * np.log2(2*np.pi*dprime*(1-dprime)*N) 
        term4 = 1/N * np.log2(8*np.pi*np.e*max(d*(1-d), 1/6)*N)
        term5 = np.log2(np.e) / (N**2 * 3 * min(dprime, 1-dprime))
        val = term1 + term2 + term3 + term4 + term5
        if val < min_val:
            min_val = val
            min_val_dprime = dprime
            min_val_term1 = term1
            min_val_term2 = term2
            min_val_term3 = term3
            min_val_term4 = term4
            min_val_term5 = term5
    return min_val, min_val_dprime, min_val_term1, min_val_term2, min_val_term3, min_val_term4, min_val_term5

In [48]:
d_vals = np.linspace(0,1,100)[1:-1]
term1s = np.array([get_min_vals(d)[2] for d in d_vals])
term2s = np.array([get_min_vals(d)[3] for d in d_vals])
term3s = np.array([get_min_vals(d)[4] for d in d_vals])
term4s = np.array([get_min_vals(d)[5] for d in d_vals])
term5s = np.array([get_min_vals(d)[6] for d in d_vals])

In [49]:
plt.plot(d_vals, term1s, label='t1')
plt.plot(d_vals, term2s, label='t2')
plt.plot(d_vals, term3s, label='t3')
plt.plot(d_vals, term4s, label='t4')
plt.plot(d_vals, term5s, label='t5')
plt.legend()